In [47]:
%pip install azure-storage-blob
%pip install azure-search-documents
%pip install python-dotenv


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [48]:
from azure.storage.blob import BlobServiceClient, generate_blob_sas, BlobSasPermissions
from datetime import datetime, timedelta
import requests
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from dotenv import load_dotenv
import os
from datetime import datetime, timedelta, timezone

In [49]:
# Load environment variables from .env file
load_dotenv()

True

In [51]:
blob_service_client = BlobServiceClient.from_connection_string(connection_string)
container_client = blob_service_client.get_container_client(container_name)

In [52]:
blob_list = container_client.list_blobs()
blob_map = {}

for blob in blob_list:
    blob_name = blob.name
    sas_token = generate_blob_sas(blob_service_client.account_name,
                                  container_name,
                                  blob_name,
                                  account_key=blob_service_client.credential.account_key,
                                  permission=BlobSasPermissions(read=True),
                                  expiry=datetime.now(timezone.utc) + timedelta(days=100)
                                )
    sas_url = f"https://{blob_service_client.account_name}.blob.core.windows.net/{container_name}/{blob_name}?{sas_token}"
    blob_map[os.path.splitext(blob_name)[0]] = sas_url
    
print(blob_map)    
    

{'Benchmark Barometer - Monthly FTSE Canada Bond Index Review - Apr23_page_1': 'https://aiexperimentpoc.blob.core.windows.net/fixedincomepdf-pages/Benchmark Barometer - Monthly FTSE Canada Bond Index Review - Apr23_page_1.pdf?se=2024-08-23T11%3A43%3A34Z&sp=r&sv=2024-05-04&sr=b&sig=BXbCJOnY/c78fVb1ga57ZKIHzH/15NkkMWWCyjaK6Ks%3D', 'Benchmark Barometer - Monthly FTSE Canada Bond Index Review - Apr23_page_10': 'https://aiexperimentpoc.blob.core.windows.net/fixedincomepdf-pages/Benchmark Barometer - Monthly FTSE Canada Bond Index Review - Apr23_page_10.pdf?se=2024-08-23T11%3A43%3A34Z&sp=r&sv=2024-05-04&sr=b&sig=6jIWjbxpDO7GZLZWaIMbKKMiTXO2IVofgLOaNnao828%3D', 'Benchmark Barometer - Monthly FTSE Canada Bond Index Review - Apr23_page_11': 'https://aiexperimentpoc.blob.core.windows.net/fixedincomepdf-pages/Benchmark Barometer - Monthly FTSE Canada Bond Index Review - Apr23_page_11.pdf?se=2024-08-23T11%3A43%3A34Z&sp=r&sv=2024-05-04&sr=b&sig=jLEWrXN9Yt7lhCD3kgkCOsIH2GeC4gt8xNsBwiDN%2B6w%3D', 'Be

In [53]:
map_count = len(blob_map)
print(map_count)

134


In [54]:
def update_blob_storage_url(chunk_id, blob_url):
 
    base_url=f"https://{service_name}.search.windows.net/indexes/{index}/docs/index?api-version=2020-06-30"
    headers = {"Content-Type": "application/json", "api-key": key}
    body = {
    "value": [
        {
            "@search.action": "merge",
            "chunk_id": chunk_id,
            "url": blob_url,
            # "need_blob_url": "true"
        }
    ]
    }
    requests.post(url = base_url, headers = headers, json = body)
 
 

In [55]:
search_client = SearchClient(service_endpoint, index, AzureKeyCredential(key))

In [56]:
results = search_client.search(search_text="*")
for result in results:
    chunk_id = result['chunk_id']
    title = os.path.splitext(result['title'])[0]
    blob_url = blob_map.get(title)
    update_blob_storage_url(chunk_id, blob_url)
    print(f'updating {title}: {blob_map.get(title)}')

updating Benchmark Barometer - Monthly FTSE Canada Bond Index Review - Sep23_page_2: https://aiexperimentpoc.blob.core.windows.net/fixedincomepdf-pages/Benchmark Barometer - Monthly FTSE Canada Bond Index Review - Sep23_page_2.pdf?se=2024-08-23T11%3A43%3A34Z&sp=r&sv=2024-05-04&sr=b&sig=q0ofdbovCKLNTF14wjG/h61Apup4Sq6Cf1%2BgevMSTp0%3D
updating Benchmark Barometer - Monthly FTSE Canada Bond Index Review - Sep23_page_4: https://aiexperimentpoc.blob.core.windows.net/fixedincomepdf-pages/Benchmark Barometer - Monthly FTSE Canada Bond Index Review - Sep23_page_4.pdf?se=2024-08-23T11%3A43%3A34Z&sp=r&sv=2024-05-04&sr=b&sig=MH%2BVmdzzdpO0zpgF7/LX/NZVXcPfHlVapStbLDX6Mmw%3D
updating Benchmark Barometer - Monthly FTSE Canada Bond Index Review - Sep23_page_8: https://aiexperimentpoc.blob.core.windows.net/fixedincomepdf-pages/Benchmark Barometer - Monthly FTSE Canada Bond Index Review - Sep23_page_8.pdf?se=2024-08-23T11%3A43%3A34Z&sp=r&sv=2024-05-04&sr=b&sig=CWK1ZHcDus9wYkh7AodZhQFtqUgwD8kwVgEDk9YXl